<p style="text-align: center;"><img src="https://upload.wikimedia.org/wikipedia/pt/1/1f/ITA_logo.png" alt="Resultado de imagem para logo ita" width="209" height="79" /></p>
<p style="text-align: center;"><span style="font-size: 14pt;">Instituto Tecnol&oacute;gico de Aeron&aacute;utica</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;">P&oacute;s-Gradua&ccedil;&atilde;o em Engenharia Eletr&ocirc;nica e Computa&ccedil;&atilde;o</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Orientador:</strong> Paulo Marcelo Tasinaffo</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Co-orientador:</strong> Rodrigo Arnaldo Scarpel</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Aluna:</strong> Bruna dos Santos Almeida</span></p>


## Import das Bibliotecas

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
#from keras.utils import to_categorical
from google.colab import drive
from sklearn.utils import class_weight

# Deep Learning Libraries
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Import do Dataset

In [2]:
drive_path = "/content/gdrive/My Drive/Colab/Thesis/"
folder = "data/"
filename = "data_synthetic_"
folder_x_y = drive_path + folder + "train_test/"

In [3]:
data_fifty_features = list(range(2,37,3))
data_synthetic_list = []
count = 1
for item in data_fifty_features:
  dict_data = {}
  dict_data['key'] = item 

  print("Processing: "+str(count)+ " of "+str(len(data_fifty_features)))  
  print("Openning data train: "+str(item))
  count = count + 1
  data_train_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_train.csv')
  data_train = pd.read_csv(data_train_path).drop("Unnamed: 0", axis=1)
  dict_data['train'] = data_train

  data_train_y = data_train['target']
  data_train_x = data_train.iloc[:,:-1]
  
  dict_data['train_y'] = keras.utils.to_categorical(data_train_y)
  dict_data['train_x'] = StandardScaler().fit_transform(data_train_x)
  
  #print("Openning data test: "+str(item))
  data_test_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_test.csv')
  data_test = pd.read_csv(data_test_path).drop("Unnamed: 0", axis=1)
  dict_data['test'] = data_test

  dict_data['test_y'] = data_test['target']
  dict_data['test_x']  = data_test.iloc[:,:-1]
  
  #print("Salve ratio and early_stopping_monitor: "+str(item))
  non_default = len(data_train[data_train['target']==0])
  default = len(data_train[data_train['target']==1])
  ratio = float(default/(non_default+default))
  dict_data['ratio'] = ratio

  #n_cols = data_train_x.shape[1]
  dict_data['early_stopping_monitor'] = EarlyStopping(patience=10)
  class_weight = {0:ratio, 1:1-ratio}
  dict_data['class_weight'] = class_weight
  
  data_synthetic_list.append(dict_data)

Processing: 1 of 12
Openning data train: 2
Processing: 2 of 12
Openning data train: 5
Processing: 3 of 12
Openning data train: 8
Processing: 4 of 12
Openning data train: 11
Processing: 5 of 12
Openning data train: 14
Processing: 6 of 12
Openning data train: 17
Processing: 7 of 12
Openning data train: 20
Processing: 8 of 12
Openning data train: 23
Processing: 9 of 12
Openning data train: 26
Processing: 10 of 12
Openning data train: 29
Processing: 11 of 12
Openning data train: 32
Processing: 12 of 12
Openning data train: 35


# Artificial Neural Network Model

## Metrics

In [4]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')
]

# Models

In [5]:
def model_training(item, data_fifty_features,  data_train_x, data_train_y, data_test_x, data_test_y, 
                   class_weight, model, data_training, n_layer, 
                   nodes_from_1, nodes_from_2, nodes_from_3):
    model_total_iter = []
    for i in range(25):
      print("Run Iteration: "+str(i+1)+" in dataset "+str(item+1))
      model_iter = {}
      model_iter['key'] = i+1

      optimizer = tf.keras.optimizers.Adadelta(learning_rate=1)
      batch_size = 2000
      EPOCHS = 200
      early_stopping_monitor = EarlyStopping(patience=10)

      model.compile(optimizer=optimizer, 
                    loss='binary_crossentropy', 
                    metrics=METRICS)
      historic = model.fit(data_train_x, 
                            data_train_y,
                            epochs=EPOCHS, 
                            validation_split=0.25, 
                            callbacks = [early_stopping_monitor], 
                            class_weight=class_weight,
                            batch_size = batch_size)
      model_iter['historic'] = historic.history

      loss_test, accuracy_test, precision_test, recall_test, auc_test, tp_test, fp_test, tn_test, fn_test = model.evaluate(data_test_x, data_test_y, batch_size=128)

      new_row = {'layers': n_layer, 
                 'nodes_from_1': nodes_from_1, 
                 'nodes_from_2': nodes_from_2, 
                 'nodes_from_3': nodes_from_3, 
                 'iteration': i+1, 
                 'dataset': data_fifty_features[item],
                 'accuracy': historic.history['accuracy'], 
                 'val_accuracy': historic.history['val_accuracy'],
                 'loss': historic.history['loss'], 
                 'val_loss': historic.history['val_auc'],
                 'auc': historic.history['auc'], 
                 'val_auc': historic.history['val_auc'],
                 'precision': historic.history['precision'],
                 'val_precision':historic.history['val_precision'],
                 'recall': historic.history['recall'], 
                 'val_recall': historic.history['val_recall'],
                 'tp': historic.history['tp'], 
                 'val_tp': historic.history['val_tp'], 
                 'fp': historic.history['fp'], 
                 'val_fp': historic.history['val_fp'], 
                 'tn': historic.history['tn'], 
                 'val_tn': historic.history['val_tn'],
                 'fn': historic.history['fn'],
                 'val_fn': historic.history['val_fn'],
                 'loss_test': loss_test,
                 'accuracy_test': accuracy_test,
                 'precision_test': precision_test,
                 'recall_test': recall_test,
                 'auc_test': auc_test,
                 'tp_test': tp_test,
                 'fp_test': fp_test,
                 'tn_test': tn_test,
                 'fn_test': fn_test}

      data_training = data_training.append(new_row, ignore_index=True)
        
      model_total_iter.append(model_iter)
    return model_total_iter, data_training

## 50 Features

### 1 Layer

In [6]:
def features_50_model_1_hidden_layer():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(50,)))
    model.add(tf.keras.layers.Dense(33))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 2 Layers

In [7]:
def features_50_model_2_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(50,)))
    model.add(tf.keras.layers.Dense(33))
    model.add(tf.keras.layers.Dense(22))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 3 Layers

In [8]:
def features_50_model_3_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(50,)))
    model.add(tf.keras.layers.Dense(33))
    model.add(tf.keras.layers.Dense(22))
    model.add(tf.keras.layers.Dense(15))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### Run Models

In [9]:
def training(item, data_fifty_features, n_layer, nodes_from_1, nodes_from_2, nodes_from_3, model, 
             data_synthetic_list, data_training):

    data_train_x = data_synthetic_list[item]['train_x']
    data_train_y = data_synthetic_list[item]['train_y']

    data_test_x = data_synthetic_list[item]['test_x']
    data_test_y = data_synthetic_list[item]['test_y']

    data_test_y = pd.DataFrame(data=data_test_y)
    data_test_y.rename(columns = {'0':'target'}, inplace = True)
    data_test_y[0] = 0
    data_test_y[1] = 0
    data_test_y.loc[data_test_y['target'] == 0, 0] = 1
    data_test_y.loc[data_test_y['target'] == 1, 1] = 1
    data_test_y = data_test_y.drop('target', axis = 1)

    class_weight = data_synthetic_list[item]['class_weight']

    model_total_iter, data_training = model_training(item, data_fifty_features, data_train_x, data_train_y, 
                                   data_test_x, data_test_y, 
                                   class_weight, model, 
                                   data_training, n_layer, 
                                   nodes_from_1, nodes_from_2, nodes_from_3)
    
    data_synthetic_list[item]['training'] = model_total_iter

    return data_synthetic_list, data_training

In [10]:
file_name = "data_training_result_50.csv"
training_result_path = drive_path + folder + file_name
training_result_path

'/content/gdrive/My Drive/Colab/Thesis/data/data_training_result_50.csv'

In [11]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 
               'val_loss', 'auc', 'val_auc','precision', 'val_precision', 
               'recall', 'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 
               'val_tn', 'fn', 'val_fn','loss_test', 'accuracy_test',
               'precision_test', 'recall_test', 'auc_test','tp_test',
               'fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 1
  nodes_from_1 = 33
  nodes_from_2 = None
  nodes_from_3 = None
  model = features_50_model_1_hidden_layer()

  data_synthetic_list, data_training = training(item, data_fifty_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

A saída de streaming foi truncada nas últimas 5000 linhas.
3/3 [==============================] - 0s 19ms/step - loss: 0.2812 - accuracy: 0.6844 - precision: 0.6844 - recall: 0.6844 - auc: 0.7319 - tp: 3850.0000 - fp: 1775.0000 - tn: 3850.0000 - fn: 1775.0000 - val_loss: 0.6390 - val_accuracy: 0.6565 - val_precision: 0.6565 - val_recall: 0.6565 - val_auc: 0.6969 - val_tp: 1231.0000 - val_fp: 644.0000 - val_tn: 1231.0000 - val_fn: 644.0000
Epoch 13/200
3/3 [==============================] - 0s 19ms/step - loss: 0.2812 - accuracy: 0.6860 - precision: 0.6860 - recall: 0.6860 - auc: 0.7317 - tp: 3859.0000 - fp: 1766.0000 - tn: 3859.0000 - fn: 1766.0000 - val_loss: 0.6391 - val_accuracy: 0.6565 - val_precision: 0.6565 - val_recall: 0.6565 - val_auc: 0.6970 - val_tp: 1231.0000 - val_fp: 644.0000 - val_tn: 1231.0000 - val_fn: 644.0000
Epoch 14/200
3/3 [==============================] - 0s 19ms/step - loss: 0.2811 - accuracy: 0.6884 - precision: 0.6884 - recall: 0.6884 - auc: 0.7328 - tp: 3872

In [12]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 2
  nodes_from_1 = 33
  nodes_from_2 = 22
  nodes_from_3 = None
  model = features_50_model_2_hidden_layers()

  data_synthetic_list, data_training = training(item, data_fifty_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

A saída de streaming foi truncada nas últimas 5000 linhas.
3/3 [==============================] - 0s 22ms/step - loss: 0.2813 - accuracy: 0.6862 - precision: 0.6862 - recall: 0.6862 - auc: 0.7317 - tp: 3860.0000 - fp: 1765.0000 - tn: 3860.0000 - fn: 1765.0000 - val_loss: 0.6384 - val_accuracy: 0.6608 - val_precision: 0.6608 - val_recall: 0.6608 - val_auc: 0.6981 - val_tp: 1239.0000 - val_fp: 636.0000 - val_tn: 1239.0000 - val_fn: 636.0000
Epoch 7/200
3/3 [==============================] - 0s 21ms/step - loss: 0.2813 - accuracy: 0.6889 - precision: 0.6889 - recall: 0.6889 - auc: 0.7329 - tp: 3875.0000 - fp: 1750.0000 - tn: 3875.0000 - fn: 1750.0000 - val_loss: 0.6389 - val_accuracy: 0.6571 - val_precision: 0.6571 - val_recall: 0.6571 - val_auc: 0.6972 - val_tp: 1232.0000 - val_fp: 643.0000 - val_tn: 1232.0000 - val_fn: 643.0000
Epoch 8/200
3/3 [==============================] - 0s 21ms/step - loss: 0.2811 - accuracy: 0.6844 - precision: 0.6844 - recall: 0.6844 - auc: 0.7321 - tp: 3850.0

In [13]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 3
  nodes_from_1 = 33
  nodes_from_2 = 22
  nodes_from_3 = 15
  model = features_50_model_3_hidden_layers()

  data_synthetic_list, data_training = training(item, data_fifty_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)
  

A saída de streaming foi truncada nas últimas 5000 linhas.
3/3 [==============================] - 0s 28ms/step - loss: 0.2814 - accuracy: 0.6885 - precision: 0.6885 - recall: 0.6885 - auc: 0.7328 - tp: 3873.0000 - fp: 1752.0000 - tn: 3873.0000 - fn: 1752.0000 - val_loss: 0.6386 - val_accuracy: 0.6560 - val_precision: 0.6560 - val_recall: 0.6560 - val_auc: 0.6973 - val_tp: 1230.0000 - val_fp: 645.0000 - val_tn: 1230.0000 - val_fn: 645.0000
Epoch 11/200
3/3 [==============================] - 0s 30ms/step - loss: 0.2813 - accuracy: 0.6841 - precision: 0.6841 - recall: 0.6841 - auc: 0.7316 - tp: 3848.0000 - fp: 1777.0000 - tn: 3848.0000 - fn: 1777.0000 - val_loss: 0.6382 - val_accuracy: 0.6581 - val_precision: 0.6581 - val_recall: 0.6581 - val_auc: 0.6984 - val_tp: 1234.0000 - val_fp: 641.0000 - val_tn: 1234.0000 - val_fn: 641.0000
Epoch 12/200
3/3 [==============================] - 0s 28ms/step - loss: 0.2812 - accuracy: 0.6868 - precision: 0.6868 - recall: 0.6868 - auc: 0.7333 - tp: 3863